In [10]:
import os
import csv
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    StratifiedKFold,
)
from sklearn.linear_model import LinearRegression,LogisticRegression
import statsmodels.api as sm
from collections import Counter
import warnings
import re 
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import Ridge
warnings.filterwarnings("ignore", category=ConvergenceWarning)




In [17]:
from sklearn.preprocessing import OneHotEncoder as _OneHotEncoder

# Wrapper de compatibilité par composition
class OneHotEncoderCompat:
    def __init__(self, *args, **kwargs):
        if "sparse" in kwargs and "sparse_output" not in kwargs:
            kwargs["sparse_output"] = kwargs.pop("sparse")
        self._enc = _OneHotEncoder(*args, **kwargs)

    def fit(self, X, y=None):
        return self._enc.fit(X, y)

    def transform(self, X):
        return self._enc.transform(X)

    def fit_transform(self, X, y=None):
        return self._enc.fit_transform(X, y)

    def get_feature_names_out(self, *args, **kwargs):
        return self._enc.get_feature_names_out(*args, **kwargs)

    def __getattr__(self, name):
        return getattr(self._enc, name)

import neurocombat_sklearn
neurocombat_sklearn.neurocombat_sklearn.OneHotEncoder = OneHotEncoderCompat
from neurocombat_sklearn import CombatModel

import neurocombat_sklearn.neurocombat_sklearn as ncs

print("OneHotEncoder patché ? ->", ncs.OneHotEncoder)


OneHotEncoder patché ? -> <class '__main__.OneHotEncoderCompat'>


In [27]:
# CONFIG
labels_path = "/neurospin/dico/rmenasria/Runs/03_main/Input/ABCD/prematurity_labels_true_classes.csv"
base_path  = "/neurospin/dico/data/deep_folding/current/models/Champollion_V1_after_ablation/embeddings/ABCD_embeddings/"


cognition_coefs = "/neurospin/dico/rmenasria/Runs/03_main/Output/final/prematurity/ABCD_prematurity_cog_directions.csv"

labels_df = pd.read_csv(labels_path, low_memory=False)
labels_df['src_subject_id'] = labels_df['src_subject_id'].str.replace("_","")


In [19]:
def load_embeddings(region):
    
    for file in os.listdir(base_path):
            if file.startswith(region) and file.endswith(".csv"):
                embedding_file = file
                break
            
    if embedding_file is None:
        raise FileNotFoundError(f"No embedding file found for region: {region}")

    print(f"Using embedding file: {embedding_file}")

    emb_path = os.path.join(base_path, embedding_file)
    emb_df = pd.read_csv(emb_path)
    emb_df['ID_clean'] = (
        emb_df['ID'].astype(str)
        .str.replace(r"^sub-", "", regex=True)
        .str.replace("_", "", regex=False)
    )
    return emb_df
    

In [20]:
def define_sex_class_mapping(sex_series):


    unique_sex = sorted(sex_series.unique())

    #print("unique sites :", unique_sex)

    mapping = {1.0: 0, 2.0:1, 3.0 : 1}
    
    def encoder_fn(site):
        return mapping[site]
    
    return mapping, encoder_fn

def define_scan_age_mapping(scan_ages_series): 


    unique_scan_ages = sorted(scan_ages_series.unique())



    scaler = StandardScaler()
    scan_ages =np.array(scan_ages_series).reshape(-1 ,1)
    scaler.fit(scan_ages)
    #print("unique_scan_ages :",unique_scan_ages)

    def encoder_fn(scan_age):
        return scaler.transform(np.array([[scan_age]])).item()

    
    return scaler, encoder_fn

In [21]:
def set_confound_df(label_counfounds_path = "/neurospin/dico/rmenasria/Runs/03_main/Input/ABCD/all_labels_clean_abcd_new_classes.csv",income_df_path = "/neurospin/dico/rmenasria/Runs/03_main/Input/ABCD/income.csv"):
    
    counfounds_df = pd.read_csv(label_counfounds_path)
    counfounds_df = pd.get_dummies(counfounds_df,columns = ['site_id_l'],prefix='',prefix_sep='',drop_first=True)
    income_df = pd.read_csv(income_df_path)
    income_df['src_subject_id_clean'] = income_df['src_subject_id'].str.replace("_","")
    #keep only necessary columns
    income_df = income_df[['src_subject_id_clean','income_continuous','missing_income']]


    mapping, encode_site = define_sex_class_mapping(counfounds_df['demo_sex_v2'])
    counfounds_df['sex'] = counfounds_df['demo_sex_v2'].apply(encode_site)
    counfounds_df.drop(columns=['demo_sex_v2'],inplace=True)


    counfounds_df["interview_age"].fillna(115,inplace =True)

    scaler, encode_scan_age = define_scan_age_mapping(counfounds_df["interview_age"])

    counfounds_df['scan_age']= counfounds_df['interview_age'].apply(encode_scan_age)

    counfounds_df = counfounds_df.merge(income_df,left_on='src_subject_id_clean',how='left',right_on='src_subject_id_clean')


    # Filter to keep only necessary columns
    columns_to_keep = ['scan_age','sex','src_subject_id_clean','income_continuous','missing_income'] + [col for col in counfounds_df.columns if col.startswith('site')]
    counfounds_df = counfounds_df[columns_to_keep]

    #print("counfounds_df columns :",counfounds_df.columns) 

    return counfounds_df


set_confound_df()

/tmp/ipykernel_300787/1431375659.py:3: DtypeWarning: Columns (307,338,352,385,397,405,436,443) have mixed types. Specify dtype option on import or set low_memory=False.
  counfounds_df = pd.read_csv(label_counfounds_path)
/tmp/ipykernel_300787/1431375659.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  counfounds_df["interview_age"].fillna(115,inplace =True)


,scan_age,sex,src_subject_id_clean,income_continuous,missing_income,site02,site03,site04,site05,site06,...,site13,site14,site15,site16,site17,site18,site19,site20,site21,site22
0,1.608618,1,NDARINV003RTV85,8.0,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,0.939494,0,NDARINV007W6H7B,10.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,-0.934053,0,NDARINV00BD7VDC,10.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0.671845,0,NDARINV00HEV6HB,8.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,-1.201702,0,NDARINV00J52GPG,6.0,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9980,-1.201702,1,NDARINVZZLZCKAY,9.0,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9981,-0.800228,1,NDARINVZZPKBDAC,10.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9982,0.270370,1,NDARINVZZZ2ALR6,10.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9983,-1.469352,1,NDARINVZZZNB0XC,3.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
def _derive_site_from_onehot(df, pattern=r'^site\d{2}$', missing_label='site_missing'):
    site_cols = [c for c in df.columns if re.match(pattern, c)]
    if not site_cols:
        return None, []
    arg = df[site_cols].astype(float).values
    ix = np.argmax(arg, axis=1)
    site_series = pd.Series(
        [site_cols[i] if (arg[row, i] == 1.0) else missing_label
         for row, i in enumerate(ix)],
        index=df.index, name='site_cat'
    )
    mask_all_zero = (arg.sum(axis=1) == 0)
    site_series.loc[mask_all_zero] = missing_label
    return site_series, site_cols

def _collapse_rare_sites(site_array, min_size):
    counts = Counter(site_array)
    return np.array([s if counts[s] >= min_size else "site_other" for s in site_array])

def prepare_cv_data(
    embeddings_df,
    labels_df,
    confounds_df,
    id_emb_col='ID_clean',
    id_label_col='src_subject_id',
    id_conf_col='src_subject_id_clean',
    confounds_for_strat=[],
    confounds_for_resid=None,
    threshold="28-32",
    dim_regex=r'^dim',
    n_splits=5,
    site_onehot_regex=r'^site\d{2}$',
    min_site_size=50  
):
    emb = embeddings_df.copy()
    lab = labels_df.copy()
    cf  = confounds_df.copy()

    emb[id_emb_col]   = emb[id_emb_col].astype(str)
    lab[id_label_col] = lab[id_label_col].astype(str)
    cf[id_conf_col]   = cf[id_conf_col].astype(str)

    merged = emb.merge(lab, left_on=id_emb_col, right_on=id_label_col, how='inner')
    df = merged.merge(cf, left_on=id_emb_col, right_on=id_conf_col, how='inner', suffixes=('','_cf'))

    # cible binaire
    df = df[df['prem_class'].isin(["<28","28-32","32-37"])].copy()
    df['y'] = (df['prem_class'].isin(["<28","28-32","32-37"])).astype(int)

    # reconstruire l’étiquette de site à partir du one-hot
    site_cat, site_cols = _derive_site_from_onehot(df, pattern=site_onehot_regex)
    if site_cat is None:
        raise ValueError(f"Aucune colonne de site détectée avec le regex {site_onehot_regex}.")
    df['site_cat'] = site_cat.astype(str)

    # collapse des sites trop petits
    if min_site_size is None:
        min_site_size = n_splits  # par défaut : au moins n_splits sujets
    df['site_cat'] = _collapse_rare_sites(df['site_cat'].values, min_site_size)

    # X / y / ids
    X_all_df = df.filter(regex=dim_regex)
    X_all = X_all_df.to_numpy(dtype=float)
    y_all = df['y'].to_numpy(dtype=int)
    ids_all = df[id_label_col].astype(str).to_numpy()

    # confounds pour résidualisation
    if confounds_for_resid is None:
        confounds_for_resid = confounds_df.columns.tolist()
        if id_conf_col in confounds_for_resid:
            confounds_for_resid.remove(id_conf_col)
    confounds_for_resid = [c for c in confounds_for_resid if c not in site_cols and c != 'site_cat']
    confounds_resid_df = df[confounds_for_resid].copy()

    # stratification : site uniquement (y peut être ignoré si tu n’as que des prémas)
    confounds_for_strat = list(confounds_for_strat) if confounds_for_strat is not None else []
    if 'site_cat' not in confounds_for_strat:
        confounds_for_strat.append('site_cat')

    stratify_labels = df['site_cat'].astype(str).to_numpy()

    # count occurrences
    counts = Counter(stratify_labels)
    print("Occurrences par strata pour stratification :", counts)

    return {
        'df_complete': df,
        'X_all_df': X_all_df,
        'X_all': X_all,
        'y_all': y_all,
        'ids_all': ids_all,
        'stratify_labels': stratify_labels,
        'confounds_resid_df': confounds_resid_df,
        'site_array': df['site_cat'].astype(str).to_numpy(),
        'site_onehot_cols': site_cols,
    }


In [24]:
def residualize_in_folds_from_prep_combat_final(
    prep,
    confounds_list=None,
    id_label_col='src_subject_id',
    id_emb_col='ID_clean',
    dim_regex=r'^dim',
    n_splits=5,
    random_state=42,
    use_statsmodels=True,
    use_combat=True,
):
    merged_df = prep['df_complete'].copy()
    X_all = prep['X_all']; y_all = prep['y_all']; ids_all = prep['ids_all']
    stratify_labels = prep['stratify_labels']
    confounds_df = prep['confounds_resid_df']

    # confounds list (déjà propres depuis prepare_cv_data) 
    if confounds_list is None:
        confounds_list = list(confounds_df.columns)  # aligné aux sujets filtrés
        # retire les id-like si présents
        confounds_list = [c for c in confounds_list if not re.search(r'(src|id)', c, flags=re.I)]

    # colonnes site à exclure de l'OLS (toutes les one-hot + site_cat)
    site_onehot_cols = [c for c in merged_df.columns if re.match(r'^site\d{2}$', c)]
    site_like_cols = set(site_onehot_cols + (['site_cat'] if 'site_cat' in merged_df.columns else []))
    confounds_list = [c for c in confounds_list if c not in site_like_cols]

    # split confounds en continu / discret (si tu en as besoin pour ComBat)
    preferred_cont = ['scan_age', 'income_continuous']
    preferred_disc = ['missing_income', 'sex']
    confounds_continuous = [c for c in preferred_cont if c in merged_df.columns and c in confounds_list]
    confounds_discrete  = [c for c in preferred_disc if c in merged_df.columns and c in confounds_list]
    remaining = [c for c in confounds_list if c not in (confounds_continuous + confounds_discrete)]
    confounds_continuous += remaining  # par défaut en continu

    # site/batch pour ComBat
    if use_combat:
        if 'site_array' in prep:
            site_series = pd.Series(prep['site_array'], index=merged_df.index).astype(str)
        else:
            # fallback: reconstruire depuis le one-hot si jamais
            if not site_onehot_cols:
                raise RuntimeError("Aucune info de site trouvée (ni site_array, ni siteXX).")
            ix = np.argmax(merged_df[site_onehot_cols].astype(float).values, axis=1)
            site_series = pd.Series([site_onehot_cols[i] for i in ix], index=merged_df.index, dtype=str)

        le = LabelEncoder()
        batch_all = le.fit_transform(site_series.values)  # ex: 'site02' -> 0..K-1
    else:
        batch_all = None

    # dims
    dim_cols = merged_df.filter(regex=dim_regex).columns.tolist()

    # K-fold (strat sur y||site_cat||confounds binned) 
    outer = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    folds_residualized = []
    id_to_y = dict(zip(ids_all.astype(str), y_all.astype(int)))

    for fold_i, (train_idx, test_idx) in enumerate(outer.split(X_all, stratify_labels)):
        train_ids = ids_all[train_idx].astype(str); test_ids = ids_all[test_idx].astype(str)

        merged_df[id_emb_col] = merged_df[id_emb_col].astype(str)
        train_mask = merged_df[id_emb_col].isin(train_ids)
        test_mask  = merged_df[id_emb_col].isin(test_ids)

        X_train_emb_df = merged_df.loc[train_mask, dim_cols].astype(float)
        X_test_emb_df  = merged_df.loc[test_mask,  dim_cols].astype(float)

        # ComBat (fit sur train, transform sur test) 
        if use_combat:
            # covariables continues (remplissage par stats train)
            if len(confounds_continuous) > 0:
                cont_train_df = merged_df.loc[train_mask, confounds_continuous].astype(float).copy()
                cont_test_df  = merged_df.loc[test_mask,  confounds_continuous].astype(float).copy()
                cont_train_df = cont_train_df.fillna(cont_train_df.median())
                cont_test_df  = cont_test_df.fillna(cont_train_df.median())
                continuous_train_arr = cont_train_df.values
                continuous_test_arr  = cont_test_df.values
            else:
                continuous_train_arr = None; continuous_test_arr = None

            # covariables discrètes
            if len(confounds_discrete) > 0:
                disc_train_df = merged_df.loc[train_mask, confounds_discrete].copy()
                disc_test_df  = merged_df.loc[test_mask,  confounds_discrete].copy()
                for c in disc_train_df.columns:
                    mv = disc_train_df[c].mode().iloc[0]
                    disc_train_df[c] = disc_train_df[c].fillna(mv)
                    disc_test_df[c]  = disc_test_df[c].fillna(mv)
            else:
                disc_train_df = pd.DataFrame(index=X_train_emb_df.index)
                disc_test_df  = pd.DataFrame(index=X_test_emb_df.index)

            # préserver y (en discrète) pour éviter de l’écraser
            # y_train_list = [id_to_y[i] for i in merged_df.loc[train_mask, id_emb_col]]
            # y_test_list  = [id_to_y[i] for i in merged_df.loc[test_mask,  id_emb_col]]
            # disc_train_df = pd.concat([
            #     disc_train_df.reset_index(drop=True),
            #     pd.Series(y_train_list, name='__y_preserve')
            # ], axis=1).set_index(X_train_emb_df.index)
            # disc_test_df = pd.concat([
            #     disc_test_df.reset_index(drop=True),
            #     pd.Series(y_test_list, name='__y_preserve')
            # ], axis=1).set_index(X_test_emb_df.index)

            discrete_train_arr = disc_train_df.values if disc_train_df.shape[1] > 0 else None
            discrete_test_arr  = disc_test_df.values  if disc_test_df.shape[1] > 0 else None

            batch_train = batch_all[train_mask.values].reshape(-1, 1)
            batch_test  = batch_all[test_mask.values].reshape(-1, 1)

            # sécurité : tout site présent en test doit exister en train
            miss = np.setdiff1d(np.unique(batch_test), np.unique(batch_train))
            if miss.size:
                raise RuntimeError(f"Fold {fold_i}: site(s) uniquement en test → {miss}. "
                                   f"Assure-toi que 'site_cat' est inclus dans stratify_labels.")

            combat = CombatModel()
            X_train_h = combat.fit_transform(X_train_emb_df.values, batch_train, discrete_train_arr, continuous_train_arr)
            X_test_h  = combat.transform(X_test_emb_df.values,  batch_test,  discrete_test_arr,  continuous_test_arr)

            X_train_h_df = pd.DataFrame(X_train_h, index=X_train_emb_df.index, columns=dim_cols)
            X_test_h_df  = pd.DataFrame(X_test_h,  index=X_test_emb_df.index,  columns=dim_cols)
        else:
            X_train_h_df = X_train_emb_df.copy()
            X_test_h_df  = X_test_emb_df.copy()

        # OLS residualization (sur embeddings harmonisés) 
        conf_train_df = merged_df.loc[train_mask, confounds_list].copy() if len(confounds_list)>0 else pd.DataFrame(index=X_train_h_df.index)
        conf_test_df  = merged_df.loc[test_mask,  confounds_list].copy() if len(confounds_list)>0 else pd.DataFrame(index=X_test_h_df.index)

        if conf_train_df.shape[1] > 0:
            for c in conf_train_df.columns:
                if pd.api.types.is_numeric_dtype(conf_train_df[c]):
                    med = conf_train_df[c].median()
                    conf_train_df[c] = conf_train_df[c].fillna(med)
                    conf_test_df[c]  = conf_test_df[c].fillna(med)
                else:
                    mv = conf_train_df[c].mode().iloc[0]
                    conf_train_df[c] = conf_train_df[c].fillna(mv)
                    conf_test_df[c]  = conf_test_df[c].fillna(mv)
        else:
            conf_train_df = pd.DataFrame(index=X_train_h_df.index)
            conf_test_df  = pd.DataFrame(index=X_test_h_df.index)

        if use_statsmodels:
            import statsmodels.api as sm
            X_conf_train = sm.add_constant(conf_train_df.astype(float), has_constant='add')
            X_conf_test  = sm.add_constant(conf_test_df.astype(float),  has_constant='add')
            resid_train_df = pd.DataFrame(index=X_train_h_df.index, columns=dim_cols, dtype=float)
            resid_test_df  = pd.DataFrame(index=X_test_h_df.index,  columns=dim_cols, dtype=float)
            for col in dim_cols:
                y_tr_col = X_train_h_df[col].astype(float)
                if X_conf_train.shape[1] == 0:
                    resid_train_df[col] = y_tr_col.values
                    resid_test_df[col]  = X_test_h_df[col].astype(float).values
                else:
                    ols = sm.OLS(y_tr_col, X_conf_train).fit()
                    resid_train_df[col] = y_tr_col - ols.predict(X_conf_train)
                    resid_test_df[col]  = X_test_h_df[col].astype(float) - ols.predict(X_conf_test)
        else:
            if conf_train_df.shape[1] == 0:
                resid_train_df = X_train_h_df.copy()
                resid_test_df  = X_test_h_df.copy()
            else:
                lr = LinearRegression()
                lr.fit(conf_train_df.values, X_train_h_df.values)
                resid_train_df = pd.DataFrame(X_train_h_df.values - lr.predict(conf_train_df.values),
                                              index=X_train_h_df.index, columns=dim_cols)
                resid_test_df  = pd.DataFrame(X_test_h_df.values  - lr.predict(conf_test_df.values),
                                              index=X_test_h_df.index,  columns=dim_cols)

        # y aligné
        train_ids_present = merged_df.loc[train_mask, id_emb_col].astype(str).tolist()
        test_ids_present  = merged_df.loc[test_mask,  id_emb_col].astype(str).tolist()
        y_train = np.array([id_to_y[i] for i in train_ids_present], dtype=int)
        y_test  = np.array([id_to_y[i] for i in test_ids_present],  dtype=int)

        folds_residualized.append({
            'X_train_resid': resid_train_df.values,
            'X_test_resid' : resid_test_df.values,
            'y_train': y_train, 'y_test': y_test,
            'train_ids': train_ids_present, 'test_ids': test_ids_present
        })

    # Full-data (pour refit/visu)
    merged_df_h = merged_df.copy()
    if use_combat:
        # mêmes covariables que plus haut (fill par stats globales)
        if len(confounds_continuous) > 0:
            full_cont_df = merged_df[confounds_continuous].astype(float).copy()
            full_cont_df = full_cont_df.fillna(full_cont_df.median())
            continuous_all_arr = full_cont_df.values
        else:
            continuous_all_arr = None

        if len(confounds_discrete) > 0:
            full_disc_df = merged_df[confounds_discrete].copy()
            for c in full_disc_df.columns:
                mv = full_disc_df[c].mode().iloc[0]
                full_disc_df[c] = full_disc_df[c].fillna(mv)
        else:
            full_disc_df = pd.DataFrame(index=merged_df.index)

        # full_disc_df = pd.concat([full_disc_df.reset_index(drop=True),
        #                           pd.Series(y_all.astype(int), name='__y_preserve')], axis=1).set_index(merged_df.index)
        discrete_all_arr = full_disc_df.values if full_disc_df.shape[1] > 0 else None

        batches_full = batch_all.reshape(-1, 1)
        combat_full = CombatModel()
        X_all_h = combat_full.fit_transform(merged_df[dim_cols].values, batches_full, discrete_all_arr, continuous_all_arr)
        merged_df_h[dim_cols] = X_all_h

    # OLS full
    confounds_for_ols_full = list(confounds_list)  # déjà sans sites
    if len(confounds_for_ols_full) == 0:
        merged_df_resid = merged_df_h.copy()
    else:
        full_conf_df = merged_df.loc[:, confounds_for_ols_full].copy()
        for c in full_conf_df.columns:
            if pd.api.types.is_numeric_dtype(full_conf_df[c]):
                full_conf_df[c] = full_conf_df[c].fillna(full_conf_df[c].median())
            else:
                full_conf_df[c] = full_conf_df[c].fillna(full_conf_df[c].mode().iloc[0])
        lr_full = LinearRegression().fit(full_conf_df.values, merged_df_h[dim_cols].values)
        X_all_resid = merged_df_h[dim_cols].values - lr_full.predict(full_conf_df.values)
        merged_df_resid = merged_df_h.copy(); merged_df_resid[dim_cols] = X_all_resid

    folds_residualized.append({
        'X_all_resid': merged_df_resid.filter(regex=dim_regex).values,
        'y_all': y_all, 'ids_all': ids_all, 'df_resid': merged_df_resid
    })
    return folds_residualized


In [25]:
def project_on_direction(X, coef, intercept):
    """
    Projects the data X onto the hyperplane defined by coef and intercept.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input data to be projected.
    coef : array-like, shape (n_features,)
        The coefficients defining the hyperplane.
    intercept : float
        The intercept defining the hyperplane.

    Returns
    -------
    projections : array, shape (n_samples,)
        The projections of each sample onto the hyperplane.
    """
    # Calculate the norm of the coefficient vector
    norm_coef = np.linalg.norm(coef)

    # Project each sample onto the hyperplane
    projections = (np.dot(X, coef) + intercept) / norm_coef

    return projections


In [48]:
def save_residualized_embeddings(region, output_path, labels_df, alpha=10.0):
    embeddings_df = load_embeddings(region)
    confounds_df = set_confound_df()
    preps = prepare_cv_data(embeddings_df, labels_df, confounds_df)
    folds_resids = residualize_in_folds_from_prep_combat_final(preps)

    selected_item = folds_resids[-1]
    X_all_resid = selected_item['X_all_resid']
    ids_all = selected_item['ids_all']
    y_all = selected_item['y_all']

    X_all_resid_df = pd.DataFrame(
        X_all_resid,
        index=ids_all,
        columns=preps['X_all_df'].columns
    )

    # save to csv
    X_all_resid_df.reset_index(inplace=True)
    X_all_resid_df.rename(columns={'index': 'ID_clean'}, inplace=True)
    
    out_csv = output_path + f"_{region}_resid.csv"
    X_all_resid_df.to_csv(out_csv, index=False)

    print(f"Saved residualized embeddings for region {region} to {out_csv}")

    # --- fit final ridge model on residualized embeddings ---
    final_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge(alpha=alpha))
    ])
    final_pipe.fit(X_all_resid, y_all)

    reg, sc = final_pipe.named_steps['regressor'], final_pipe.named_steps['scaler']
    w_x = reg.coef_.ravel() / sc.scale_
    b_x = float(reg.intercept_) - np.dot(w_x, sc.mean_)

    return {
        "X_all_resid": X_all_resid,
        "y_all": y_all,
        "ids_all": ids_all,
        "out_csv": out_csv,
        "w_x": w_x,
        "b_x": b_x
    }



In [34]:
region = "ScCal-SLi_left"
output_path = "/neurospin/dico/rmenasria/Runs/03_main/Output/embeddings_residualized/cognition"
save_residualized_embeddings(region, output_path, labels_df)

Using embedding file: ScCal-SLi_left_name07-41-43--180_embeddings.csv


/tmp/ipykernel_300787/1431375659.py:3: DtypeWarning: Columns (307,338,352,385,397,405,436,443) have mixed types. Specify dtype option on import or set low_memory=False.
  counfounds_df = pd.read_csv(label_counfounds_path)
/tmp/ipykernel_300787/1431375659.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  counfounds_df["interview_age"].fillna(115,inplace =True)


Occurrences par strata pour stratification : Counter({'site_other': 388, 'site20': 197, 'site14': 192, 'site02': 182, 'site19': 165, 'site03': 61, 'site10': 52, 'site16': 51, 'site13': 51})
Saved residualized embeddings for region ScCal-SLi_left to /neurospin/dico/rmenasria/Runs/03_main/Output/embeddings_residualized/cognition_ScCal-SLi_left_resid.csv


In [44]:
def parse_numpy_like_string(s):
    # enlève les crochets
    s = s.strip().lstrip("[").rstrip("]")
    # découpe sur les espaces
    nums = re.split(r"\s+", s.strip())
    # convertit en float
    return np.array([float(x) for x in nums if x != ""])

df_dir_28_32 = pd.read_csv(cognition_coefs)

def save_directions(region, score, output_path):
    global df_dir_28_32

    resid_path = f"/neurospin/dico/rmenasria/Runs/03_main/Output/embeddings_residualized/cognition/prema_cognition_{region}_resid.csv"
    resid_df = pd.read_csv(resid_path)

    # sélection correcte : filtrer à la fois sur région ET score
    dir_row = df_dir_28_32[(df_dir_28_32['region'] == region) & (df_dir_28_32['score'] == score)]
    if dir_row.empty:
        raise ValueError(f"No direction found for region: {region}, score: {score}")

    coef_str = dir_row['final_coef_original'].values[0]
    intercept_val = dir_row['final_intercept_original'].values[0]

    # parse coefficients
    coef = parse_numpy_like_string(coef_str)

    # intercept peut être string avec [] ou déjà un float
    if isinstance(intercept_val, str):
        intercept = parse_numpy_like_string(intercept_val).item()
    else:
        intercept = float(intercept_val)

    print("coef shape:", coef.shape)
    print("intercept:", intercept)

    # compute projections
    projections = project_on_direction(
        resid_df.filter(regex=r'^dim').values, coef, intercept
    )

    # build DataFrame sorted by projection (descending = most preterm first)
    directions_df = pd.DataFrame({
        'ID_clean': resid_df['ID_clean'],
        'projection': projections
    }).sort_values(by='projection', ascending=False)

    output_file = os.path.join(output_path, f"{region}_{score}_direction.csv")
    directions_df.to_csv(output_file, index=False)

    print(f"Saved sorted directions for region {region}, score {score} to {output_file}")


region = "STi-SOTlat_right"
output_path = "/neurospin/dico/rmenasria/Runs/03_main/Output/final_direction/prema_cognition"
save_directions(region,"nihtbx_picvocab_agecorrected", output_path)


coef shape: (32,)
intercept: 105.60995862277544
Saved sorted directions for region STi-SOTlat_right, score nihtbx_picvocab_agecorrected to /neurospin/dico/rmenasria/Runs/03_main/Output/final_direction/prema_cognition/STi-SOTlat_right_nihtbx_picvocab_agecorrected_direction.csv


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np

def check_equivalence(X_all_resid, y_all, best_alpha, coef, intercept):
    """
    Vérifie que les coefficients/intercept corrigés reproduisent bien
    les prédictions du pipeline [StandardScaler + Ridge].
    """

    # Refit pipeline standard
    pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge(alpha=best_alpha))])
    pipe.fit(X_all_resid, y_all)

    # Prédictions pipeline (avec scaler)
    y_pred_pipe = pipe.predict(X_all_resid)

    # Prédictions via coef/intercept "corrigés"
    y_pred_direct = X_all_resid @ coef + intercept

    # Comparaison
    print("Sanity check:")
    print("  Pipeline preds  -> min:", y_pred_pipe.min(), "max:", y_pred_pipe.max(), "mean:", y_pred_pipe.mean())
    print("  Direct preds    -> min:", y_pred_direct.min(), "max:", y_pred_direct.max(), "mean:", y_pred_direct.mean())
    print("  Diff (MSE):", np.mean((y_pred_pipe - y_pred_direct) ** 2))

    # Vérif stricte : doivent être quasi-identiques
    if np.allclose(y_pred_pipe, y_pred_direct, rtol=1e-6, atol=1e-6):
        print("OK : les coef/intercept corrigés sont cohérents avec le pipeline.")
    else:
        print("Attention : il y a une différence, vérifier la sauvegarde des coef/intercept.")



res = save_residualized_embeddings(region, output_path, labels_df)
check_equivalence(res["X_all_resid"], res["y_all"], 0.1, res["w_x"], res["b_x"])



Using embedding file: STi-SOTlat_right_name06-17-38--74_embeddings.csv


/tmp/ipykernel_300787/1431375659.py:3: DtypeWarning: Columns (307,338,352,385,397,405,436,443) have mixed types. Specify dtype option on import or set low_memory=False.
  counfounds_df = pd.read_csv(label_counfounds_path)
/tmp/ipykernel_300787/1431375659.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  counfounds_df["interview_age"].fillna(115,inplace =True)


Occurrences par strata pour stratification : Counter({'site_other': 388, 'site20': 197, 'site14': 192, 'site02': 182, 'site19': 165, 'site03': 61, 'site10': 52, 'site16': 51, 'site13': 51})
Saved residualized embeddings for region STi-SOTlat_right to /neurospin/dico/rmenasria/Runs/03_main/Output/final_direction/prema_cognition_STi-SOTlat_right_resid.csv
Sanity check:
  Pipeline preds  -> min: 1.0 max: 1.0 mean: 1.0
  Direct preds    -> min: 1.0 max: 1.0 mean: 1.0
  Diff (MSE): 0.0
✅ OK : les coef/intercept corrigés sont cohérents avec le pipeline.
